In [2]:
import pandas as pd

student = pd.DataFrame(data=[
    {'姓名':'徐國堂','國文':87,'英文':92,'數學':86},
    {'姓名':'王國堂','國文':92,'英文':80,'數學':90},
    {'姓名':'張國堂','國文':76,'英文':83,'數學':65}
])

student

,姓名,國文,英文,數學
0,徐國堂,87,92,86
1,王國堂,92,80,90
2,張國堂,76,83,65
